# Synthetic Events

In [ ]:
from components.ingestion.synthetic.synthetic_errors import SyntheticErrors
from components.ingestion.synthetic.synthetic_events import SyntheticEvents

## Init Component

In [ ]:
root_path = "/opt/dev"

general_config = f"{root_path}/pipe_configs/configurations/general_config.ini"
component_config = f"{root_path}/pipe_configs/configurations/synthetic_events/synth_config.ini"

In [ ]:
# Start Component
events = SyntheticEvents(general_config, component_config)
errors = SyntheticErrors(general_config, component_config)

events.spark

## Main

In [ ]:
# events.execute()
errors.execute()

In [ ]:
events.execute()
errors.read()
errors.transform()

In [ ]:
input_df = errors.input_data_objects["SyntheticEvents"].df
input_df.show()

In [ ]:
input_df.select('timestamp').distinct().sort('timestamp').show()

In [ ]:
df = errors.output_data_objects["SyntheticErrors"].df
df.cache()
df.show()

In [ ]:
df.select('timestamp').distinct().sort('timestamp').show(truncate=False)

In [ ]:
df.printSchema()

In [ ]:
errors.write()

In [ ]:
input_df.count()